In [1]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import sys
sys.path.append('../')
import prompts as p

# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

DATA_PATH = "../../Datasets/Vehice dataset/Downsampled/Test/test_sampled_few_shot.csv"
OUTPUT_PATH = "../../Datasets/Evaluations/Regression/few_shot_pipeline_4-turbo-preview.csv"

MODEL = "gpt-4-turbo-preview"
MAX_TOKENS = 15
TEMP = 0

In [2]:
df = pd.read_csv(DATA_PATH)
prompt_name = "few_shot"
prompt = p.few_shot

df['prediction'] = None
df["prompt"] = None
df["prompt_name"] = prompt_name
df["model_name"] = MODEL


print(df.head())


                                    name  year  selling_price  km_driven  \
0     Ford Figo Aspire 1.5 TDCi Titanium  2017         670000      70000   
1  Mahindra Scorpio VLX 2WD AIRBAG BSIII  2012         525000     120000   
2                 Maruti Swift Dzire VDI  2014         438999      81000   
3              Ford Figo Diesel Titanium  2010         144000      50000   
4                 Hyundai i10 Magna 1.1L  2008         185000     110000   

     fuel seller_type transmission                 owner     mileage   engine  \
0  Diesel  Individual       Manual           First Owner  25.83 kmpl  1498 CC   
1  Diesel  Individual       Manual           First Owner  12.05 kmpl  2179 CC   
2  Diesel      Dealer       Manual           First Owner   23.4 kmpl  1248 CC   
3  Diesel  Individual       Manual          Second Owner   20.0 kmpl  1399 CC   
4  Petrol  Individual       Manual  Fourth & Above Owner  19.81 kmpl  1086 CC   

   ...                                   example_prompt_

In [3]:
import re
def extract_number(text):
    # Remove commas and points
    clean_string = re.sub(r'[,.]', '', text)

    # Extract the number
    number = re.search(r'\d+', clean_string).group()

    return number


In [4]:
def execute(df, prompt, prompt_name, start = 0):
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, max_tokens=MAX_TOKENS, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, max_tokens=MAX_TOKENS, timeout=10)
    for i in range(start, len(df)):
        if(df["prediction"][i] != None and df["prediction"][i] != "error"):
            continue
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])

        #Get Few Shot Examples
        example_1 = df["example_prompt_1"][i]
        example_2 = df["example_prompt_2"][i]
        example_3 = df["example_prompt_3"][i]

        #Get Basic Data
        name = str(df["name"][i])
        year = str(df["year"][i])   
        selling_price = str(df["selling_price"][i])
        km_driven = str(df["km_driven"][i])
        fuel = str(df["fuel"][i])
        seller_type = str(df["seller_type"][i])
        transmission = str(df["transmission"][i])
        owner = str(df["owner"][i])
        mileage = str(df["mileage"][i])
        engine = str(df["engine"][i])
        max_power = str(df["max_power"][i])
        torque = str(df["torque"][i])
        seats = str(df["seats"][i])
        
        try: 
            result = chain.run(
                example_1 = example_1,
                example_2 = example_2,
                example_3 = example_3,
                name = name, 
                year = year, 
                selling_price = selling_price, 
                km_driven = km_driven, 
                fuel = fuel, 
                seller_type = seller_type,
                transmission = transmission, 
                owner = owner,
                mileage = mileage, 
                engine = engine,
                max_power = max_power, 
                torque = torque,
                seats = seats,
                callbacks=[handler])
        except Exception as e:
            print(e)
            df.loc[i, 'prediction'] =  'error'
            df.loc[i, 'prompt'] = chain.prompt.format_prompt(
                example_1 = example_1,
                example_2 = example_2,
                example_3 = example_3,
                name = name, 
                year = year, 
                selling_price = selling_price, 
                km_driven = km_driven, 
                fuel = fuel, 
                seller_type = seller_type,
                transmission = transmission, 
                owner = owner,
                mileage = mileage, 
                engine = engine,
                max_power = max_power, 
                torque = torque,
                seats = seats).text
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(
            example_1 = example_1,
            example_2 = example_2,
            example_3 = example_3,
            name = name, 
            year = year, 
            selling_price = selling_price, 
            km_driven = km_driven, 
            fuel = fuel, 
            seller_type = seller_type,
            transmission = transmission, 
            owner = owner,
            mileage = mileage, 
            engine = engine,
            max_power = max_power, 
            torque = torque,
            seats = seats).text
        if (i < 5):
            print(prompt_text)
            print("\n")
            print("selling price: " + selling_price)
            print("predicted price: " + result)
            print("\n")
        
        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))

        df.loc[i, 'prediction'] =  extract_number(result)
        df.loc[i, 'prompt'] = prompt_text
    return df


df = execute(df, prompt, prompt_name)
    



0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:86: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Example 1:
Features:
name: Ford Figo Aspire 1.5 TDCi Trend,
year: 2017,
km_driven: 35000,
fuel: Diesel,
seller_type: Individual,
transmission: Manual,
owner: First Owner,
mileage: 25.83 kmpl,
engine: 1498 CC,
max_power: 99 bhp,
torque: 215Nm@ 1750-3000rpm,
seats: 5.0

Output: "price": 700000 INR 

Example 2:
Features:
name: Ford Figo Aspire 1.5 TDCi Ambiente,
year: 2016,
km_driven: 70000,
fuel: Diesel,
seller_type: Individual,
transmission: Manual,
owner: First Owner,
mileage: 25.83 kmpl,
engine: 1498 CC,
max_power: 99 bhp,
torque: 215Nm@ 1750-3000rpm,
seats: 5.0

Output: "price": 500000 INR 

Example 3:
Features:
name: Ford Figo Aspire 1.5 TDCi Titanium,
year: 2015,
km_driven: 70000,
fuel: Diesel,
seller_type: Individual,
transmission: Manual,
owner: First Owner,
mileage: 25.83 kmpl,
engine: 1498 CC,
max_power: 99 bhp,
torque: 215Nm@ 1750-3000rpm,
seats: 5.0

Output: "price": 433000 INR 

System: Based on the provided features of a used car listed below, please predict its selling pri

In [5]:
df

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,...,example_prompt_2,example_score_2,example_index_2,example_prompt_3,example_score_3,example_index_3,prediction,prompt,prompt_name,model_name
0,Ford Figo Aspire 1.5 TDCi Titanium,2017,670000,70000,Diesel,Individual,Manual,First Owner,25.83 kmpl,1498 CC,...,Features:\nname: Ford Figo Aspire 1.5 TDCi Amb...,0.772090,2923,Features:\nname: Ford Figo Aspire 1.5 TDCi Tit...,0.801998,3341,600000,Example 1:\nFeatures:\nname: Ford Figo Aspire ...,few_shot,gpt-4-turbo-preview
1,Mahindra Scorpio VLX 2WD AIRBAG BSIII,2012,525000,120000,Diesel,Individual,Manual,First Owner,12.05 kmpl,2179 CC,...,Features:\nname: Mahindra Scorpio VLX 2WD BSIV...,0.811647,1951,Features:\nname: Mahindra Scorpio VLX 2WD AIRB...,0.856752,872,550000,Example 1:\nFeatures:\nname: Mahindra Scorpio ...,few_shot,gpt-4-turbo-preview
2,Maruti Swift Dzire VDI,2014,438999,81000,Diesel,Dealer,Manual,First Owner,23.4 kmpl,1248 CC,...,"Features:\nname: Maruti Swift Dzire VDI,\nyear...",0.600578,3986,"Features:\nname: Maruti Swift Dzire VDI,\nyear...",0.602942,2745,440000,Example 1:\nFeatures:\nname: Maruti Swift Dzir...,few_shot,gpt-4-turbo-preview
3,Ford Figo Diesel Titanium,2010,144000,50000,Diesel,Individual,Manual,Second Owner,20.0 kmpl,1399 CC,...,"Features:\nname: Ford Figo Diesel Titanium,\ny...",0.720091,4896,"Features:\nname: Ford Figo Diesel Titanium,\ny...",0.742243,356,220000,Example 1:\nFeatures:\nname: Ford Figo Diesel ...,few_shot,gpt-4-turbo-preview
4,Hyundai i10 Magna 1.1L,2008,185000,110000,Petrol,Individual,Manual,Fourth & Above Owner,19.81 kmpl,1086 CC,...,"Features:\nname: Hyundai i10 Magna 1.1L,\nyear...",0.786414,4235,"Features:\nname: Hyundai i10 Magna 1.1L,\nyear...",0.795078,3154,200000,Example 1:\nFeatures:\nname: Hyundai i10 Magna...,few_shot,gpt-4-turbo-preview
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,Honda City 1.5 S MT,2011,421000,105000,Petrol,Individual,Manual,First Owner,17.0 kmpl,1497 CC,...,"Features:\nname: Honda City 1.5 V MT,\nyear: 2...",0.776087,3520,"Features:\nname: Honda City 1.5 E MT,\nyear: 2...",0.783398,396,400000,Example 1:\nFeatures:\nname: Honda City 1.5 E ...,few_shot,gpt-4-turbo-preview
396,Maruti Swift VXI Optional,2017,600000,24522,Petrol,Dealer,Manual,First Owner,20.4 kmpl,1197 CC,...,"Features:\nname: Maruti Swift VXI Optional,\ny...",0.818000,2366,"Features:\nname: Maruti Swift VXI Optional,\ny...",0.818000,288,550000,Example 1:\nFeatures:\nname: Maruti Swift VXI ...,few_shot,gpt-4-turbo-preview
397,Maruti Vitara Brezza LDi Option,2016,640000,70000,Diesel,Individual,Manual,First Owner,24.3 kmpl,1248 CC,...,Features:\nname: Maruti Vitara Brezza LDi Opti...,0.783034,706,Features:\nname: Maruti Vitara Brezza LDi Opti...,0.820770,1282,620000,Example 1:\nFeatures:\nname: Maruti Vitara Bre...,few_shot,gpt-4-turbo-preview
398,Hyundai Xcent 1.2 CRDi E Plus,2017,450000,40000,Diesel,Individual,Manual,Second Owner,25.4 kmpl,1186 CC,...,Features:\nname: Hyundai Grand i10 1.2 CRDi Sp...,0.697778,704,Features:\nname: Hyundai Xcent 1.2 CRDi E Plus...,0.809091,3431,430000,Example 1:\nFeatures:\nname: Hyundai Grand i10...,few_shot,gpt-4-turbo-preview


In [6]:
df.to_csv(OUTPUT_PATH, index = True)